<a href="https://colab.research.google.com/github/juliewang2020/FreeRealEstate/blob/master/parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Data


In [10]:
import pandas as pd
import io
from google.colab import files
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['data.csv']))


Saving data.csv to data (1).csv


In [0]:
############## Cleaning the HTML

#get rid of ID and CreatedDate
features = df.iloc[:, 1:]
features.drop('CreatedDate', axis = 1, inplace=True)

import re

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
  cleantext = re.sub(cleanr, '', raw_html)
  return cleantext

for i in range(0, 1933):
  body_text = features.iloc[i , 2]
  body_text = cleanhtml(body_text)
  body_text = body_text.replace('\n', ' ')
  features.loc[i, 'Body'] = body_text


In [0]:
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [12]:
import nltk
from sklearn.feature_extraction import text
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

eng_contractions = ["ain't", "amn't", "aren't", "can't", "could've", "couldn't",
                    "daresn't", "didn't", "doesn't", "don't", "gonna", "gotta", 
                    "hadn't", "hasn't", "haven't", "he'd", "he'll", "he's", "how'd",
                    "how'll", "how's", "I'd", "I'll", "I'm", "I've", "isn't", "it'd",
                    "it'll", "it's", "let's", "mayn't", "may've", "mightn't", 
                    "might've", "mustn't", "must've", "needn't", "o'clock", "ol'",
                    "oughtn't", "shan't", "she'd", "she'll", "she's", "should've",
                    "shouldn't", "somebody's", "someone's", "something's", "that'll",
                    "that're", "that's", "that'd", "there'd", "there're", "there's", 
                    "these're", "they'd", "they'll", "they're", "they've", "this's",
                    "those're", "tis", "twas", "twasn't", "wasn't", "we'd", "we'd've",
                    "we'll", "we're", "we've", "weren't", "what'd", "what'll", 
                    "what're", "what's", "what've", "when's", "where'd", "where're",
                    "where's", "where've", "which's", "who'd", "who'd've", "who'll",
                    "who're", "who's", "who've", "why'd", "why're", "why's", "won't",
                    "would've", "wouldn't", "y'all", "you'd", "you'll", "you're", 
                    "you've", "'s", "s"
                     ]

nltk.download('stopwords')
nltk.download('punkt')

custom_stopwords = text.ENGLISH_STOP_WORDS.union(eng_contractions)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
import nltk
import re

def tokenize_and_stem(text, do_stem=True):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
            
    # stem filtered tokens
    stems = [stemmer.stem(t) for t in filtered_tokens]
    
    if do_stem:
        return stems
    else:
        return filtered_tokens

In [25]:
# not super pythonic, no, not at all.
# use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in range(0, 1933):
    body_text = features.iloc[i , 2]
    allwords_stemmed = tokenize_and_stem(body_text)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_and_stem(body_text, False)
    totalvocab_tokenized.extend(allwords_tokenized)  
  
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

,words
washington,washington
food,food
and,and
suppli,supply
of,of


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words=custom_stopwords,
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

tfidf_matrix = tfidf_vectorizer.fit_transform(features['Body']) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

terms = tfidf_vectorizer.get_feature_names()

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'m", 'abov', 'afterward', 'ai', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'ca', 'cri', 'dare', 'describ', 'did', 'doe', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'gon', 'got', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'let', 'll', 'mani', 'meanwhil', 'moreov', "n't", 'na', 'need', 'nobodi', 'noon', 'noth', 'nowher', 'ol', 'onc', 'onli', 'otherwis', 'ought', 'ourselv', 'perhap', 'pleas', 'sever', 'sha', 'sinc', 'sincer', 'sixti', 'somebodi', 'someon', 'someth', 'sometim', 'somewher', 'ta', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 've', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'where

(1933, 104)


In [27]:
from sklearn.cluster import KMeans
import math

num_clusters = int(math.sqrt(features.shape[0] / 2) * 1.5)

km = KMeans(n_clusters=num_clusters)

km.fit(tfidf_matrix)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=46, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [28]:
clusters = km.labels_.tolist()

features['cluster'] = clusters
features.head()

,Title,Summary,Body,Hits,AuthorID,Country_USA,Country_CAN,Country_GBR,Tag_None,Tag_National,Tag_Office,Tag_Industrial,Tag_Retail,Tag_People,Tag_Investment,Tag_Analytics,Tag_Development,Tag_Finance,Tag_Events,Tag_Multifamily,Tag_Hospitality,Tag_Company,Tag_Healthcare,Tag_Legal,Tag_Land,Tag_Lease,Tag_Sale,Tag_MixedUse,Tag_SpecialPurpose,Tag_PublicSector,Tag_CompaniesPeople,Tag_CoStarGreenReport,cluster
0,"Paramount Transportation Leases 136,000-SF Che...",Will Occupy Former Safeway Warehouse in September,"Washington Food and Supply of Maryland, Inc., ...",0,132,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,41
1,Newmark Knight Frank Boosts National Appraisal...,Exclusive: Newmark to Acquire Integra Affiliat...,Newmark Knight Frank (NKF) is finalizing the a...,32,296,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,37
2,Amazon Outgrows Seattle: Opens Search for Seco...,Tech Giant Plans to Invest More Than $5 Billio...,Amazon today is posting another unique offerin...,71,240,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,15
3,"For Amazon’s Second HQ’s Search, Bigger May Be...",Biggest US Cities Appear to Have Edge in Amazo...,While there has been no shortage of market con...,601,240,1,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,19
4,Last Call for Amazon HQ2: Today is Deadline fo...,Hundreds of Communities Expected to Pitch Prop...,Today is the last day for communities to submi...,1379,240,1,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,36


In [31]:
print("Top terms per cluster:")
print()

#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.loc[terms[ind].split(' ')].values.tolist()[0][0], end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d titles:" % i, end='')
    print()
    for title in features[features['cluster'] == i]['Title'].values.tolist():
        print(' - %s' % title)
    print() #add whitespace
    print() #add whitespace
    
print()
print()

Top terms per cluster:

Cluster 0 words: percent, markets, growth, rents, since, year,

Cluster 0 titles:
 - Job Growth Continues the Longest Streak of Employment Gains Recorded, Weekly Earnings Climb
 - Multifamily Sales On Pace of Reaching Record High for the Year
 - San Diego Apartment Sales Are Cooling
 - Office Construction Continues to Extend Outwards in the Four Major Texas Metros
 - What Impact Does Apartment Seasonality Have on Houston?
 - North Delaware Apartment Sales Climb to New Heights
 - Could Augusta, Georgia Be Underbuilt?
 - Forecast: Commercial Construction Spending Growth to Reach Decade High in 2019
 - The Music City Keeps Building Apartments
 - Orange County Economy Remains on Mostly Solid Footing
 - Multifamily Sales Activity Continues to Soar in Nashville
 - Jacksonville’s Economy Brings the Heat
 - Myrtle Beach Multifamily Rent Growth Among the Highest in the Nation
 - Rising Home Sales May Threaten Multifamily Gains in Alabama
 - Multifamily Construction Charg